### CNN 연습

In [51]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
conv2 = nn.Conv2d(32, 64, 3, padding=1)
conv2

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [6]:
pool = nn.MaxPool2d(2)
pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [10]:
out = conv1(inputs)
out.shape

torch.Size([1, 32, 28, 28])

In [11]:
out = pool(out)
out.shape

torch.Size([1, 32, 14, 14])

In [12]:
out = conv2(out)
out.shape

torch.Size([1, 64, 14, 14])

In [13]:
out = pool(out)
out.shape

torch.Size([1, 64, 7, 7])

In [15]:
(out.size(0), out.size(1), out.size(2), out.size(3))

(1, 64, 7, 7)

In [17]:
out = out.view(out.size(0), -1)
out.shape

torch.Size([1, 3136])

In [18]:
fc = nn.Linear(3136, 10)
out = fc(out)
out.shape

torch.Size([1, 10])

### CNN으로 MNIST 분류하기

In [62]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [63]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

lr = 1e-3
EPOCHS = 10
batch_size = 100

In [64]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True,)

In [26]:
mnist_train.data.shape

torch.Size([60000, 28, 28])

In [27]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [28]:
model = CNN().to(device)
crit = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

total_batch = len(data_loader)

In [31]:
for i in range(EPOCHS):
    losses = 0
    for x, y in data_loader:
        x, y = x.to(device), y.to(device)
        y_hat = model(x)
        loss = crit(y_hat, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses += loss
    real_loss = losses / total_batch
    print("EPOCHS: {:2d} | loss: {:.4f}".format(i, real_loss))

EPOCHS:  0 | loss: 0.2217
EPOCHS:  1 | loss: 0.0619
EPOCHS:  2 | loss: 0.0464
EPOCHS:  3 | loss: 0.0371
EPOCHS:  4 | loss: 0.0314
EPOCHS:  5 | loss: 0.0263
EPOCHS:  6 | loss: 0.0219
EPOCHS:  7 | loss: 0.0187
EPOCHS:  8 | loss: 0.0159
EPOCHS:  9 | loss: 0.0125


In [42]:
with torch.no_grad():
    x_test = mnist_test.data.view(len(mnist_test),1, 28, 28).float().to(device)
    y_test = mnist_test.targets.to(device)
    y_test_hat = model(x_test)
    correct = torch.argmax(y_test_hat, 1) == y_test
    accuracy = correct.float().mean()
    print("Acuuracy: ", accuracy.item())

Acuuracy:  0.9876999855041504


### 더 깊은 모델 만들기

In [65]:
class DeepCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(128*3*3, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.LogSoftmax(-1)
        )
        
    def forward(self, x):
        out = self.layer3(self.layer2(self.layer1(x)))
        out = out.view(out.size(0), -1)
        return self.fc_layer(out)

In [66]:
model = DeepCNN().to(device)
crit = nn.NLLLoss().to(device)
optimizer = optim.Adam(model.parameters() ,lr=lr)
num_of_batch = len(data_loader)

for i in range(EPOCHS):
    losses = 0
    for x, y in data_loader:
        x, y = x.to(device), y.to(device)
        y_hat = model(x)
        loss = crit(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses += loss / num_of_batch
    print('EPOCHS: {:4d} | loss: {:.4f}'.format(i, losses))

EPOCHS:    0 | loss: 0.1912
EPOCHS:    1 | loss: 0.0455
EPOCHS:    2 | loss: 0.0312
EPOCHS:    3 | loss: 0.0249
EPOCHS:    4 | loss: 0.0185
EPOCHS:    5 | loss: 0.0172
EPOCHS:    6 | loss: 0.0114
EPOCHS:    7 | loss: 0.0126
EPOCHS:    8 | loss: 0.0099
EPOCHS:    9 | loss: 0.0087


In [69]:
with torch.no_grad():
    x_test = mnist_test.data.view(len(mnist_test),1,28,28).float().to(device)
    y_test = mnist_test.targets.to(device)
    y_test_hat = model(x_test)
    accuracy = (torch.argmax(y_test_hat, 1) == y_test).float().mean()
    print("accuracy: ", accuracy.item())

accuracy:  0.9922999739646912
